# Variational quantum eigensolver (VQE)

**Note: The TP is in english, but feel free to answer questions in french **

The purpose of this TP is to introduce a new concept in quantum algorithmics: variational quantum eigensolving (VQE).
This class of algorithms tries to tackle the following problem:


## Problem

**Input**: some Hamiltonian $H$

**Output**: the ground energy $E_0$ of $H$ (or at least an approximation of it)


You should already be familiar with the Quantum Phase Estimation algorithm (QPE) that partly solves this problem

## VQE algorithm

The approach taken in a VQE algorithm is less resource demanding than a full QPE and relies on the following structure:

- pick a family of quantum states $|\psi(\theta)\rangle$, parametrized by a collection of angles $\theta = (\theta_1, ..., \theta_k)$
- use a classical optimizer and a quantum computer to find $\theta^\star = arg min_\theta \langle \psi(\theta) | H | \psi(\theta) \rangle$.
- output $\langle \psi(\theta^\star) | H | \psi(\theta^\star) \rangle$


This type of approach assumes several things:
- first, we need an *interesting* family of quantum states $|\psi(\theta)\rangle$. In particular, we are looking for states that:
    - can be prepared efficiently. That is, we have a simple quantum circuit $C(\theta)$ such that $|\psi(\theta)\rangle = C(\theta)|0\rangle$ 
    - can approximate faithfully the ground state of $H$. That is, there exists some $\theta^*$ such that $|\psi(\theta^\star)\rangle \approx |GS\rangle$

- second, we need to be able to efficiently use a quantum computer to evaluate the quantify $\langle \psi(\theta)|H|\psi(\theta)\rangle$. This is usually enforced by having $H = \sum_i \alpha_i P_i$ where $P_i$ are Pauli operators and the sum is not too large.

## This TP

In this TP, we will consider a very standard class of Hamiltonian called Ising Hamiltonians.
The motivation being that they can naturally encode many combinatorial optimization problems.

We will pick the following convention for our Hamiltonians:

$$ H = \sum_{1 \leq i < j \leq n} J_{i, j} \sigma_z^i \sigma_z^j $$


Notice two things (that might differ from standard definitions):
- the sum runs over pairs $(i, j)$ of spins where $i < j$
- there is not *magnetic field* term, only coupling terms



Let's start with a few questions to get you familiar with what we want to achieve.

**Question 1:**

What can we say about the ground state of such an Hamitonian?

Can you devise a simple (yet very costly) classical algorithm we could use to find the ground state and its energy? (no need to implement it yet) 

**Answer**: 



In the rest of the TP, we will describe $H$ via a python map that maps pairs  $(i, j)$ to their corresponding coefficients $J_{i, j}$.

For instance, $H = \sigma_z^0\sigma_z^1 + 2 \sigma_z^1\sigma_z^2$
will be represented by the following map:


In [ ]:
H = {(0, 1): 1, (1, 2): 2}

import numpy as np

Z = np.diag([1, -1])
ZZ = np.diag([1, -1, -1, 1])
# 00 --> 1
# 01 --> -1
# 10 --> -1
# 11 --> 1

**Question 2:**

Write a python function that takes such a description of an Ising Hamiltonian and outputs its ground energy.
As mentionned in question 1, we do not expect this function to be efficient.

The function should pass the three asserts below.

In [ ]:
def exact_ground_energy(ham):
    """ TODO """

In [ ]:
# Testing
assert exact_ground_energy({(0, 1): 1, (1, 2): 2}) == -3
assert exact_ground_energy(
    {(i, i+1): 1 for i in range(10)}
) == -10
assert exact_ground_energy(
    {(0, 1): 1, (1, 2): 1, (0, 2): 1}
) == -1

## Let's write a VQE

We will now use the myqlm library to write a VQE approach to approximate the ground energy $E_0$.

### Encoding $H$


**Question 3:** Write a python function that take an Hamiltonian described by a python map and return a myqlm `Observable` object. Refer to [this section of the documentation](https://myqlm.github.io/introduction.html#observables).


In [ ]:


def to_observable(ham):
    """TODO"""


In [ ]:
# Tests
assert to_observable({(0, 1): 1, (1, 2): 2}).nbqbits == 3
assert len(to_observable({(0, 1): 1, (1, 2): 2}).terms) == 2
assert to_observable({(i, i+1): 23 for i in range(47)}).nbqbits == 48
assert len(to_observable({(i, i+1): 23 for i in range(47)}).terms) == 47

### A variational circuit

In order to run a VQE, we need a family of parametrized quantum states $|\psi(\theta)\rangle$.

As specified in the introduction, we will described these states using a family of quantum circuits $C(\theta)$ such that $|\psi(\theta)\rangle = C(\theta) |0\rangle$.

[This section of the documentation](https://myqlm.github.io/running_variational.html#variational-jobs) explains how to construct a parametrized circuit in myqlm.


We will start with a first circuit family where circuits are composed of:

- a layer of Hadamard gates (one gate per qbit)
- alternating layers:
    - a layer of RZZ rotations (as defined in [the helper notebook](./tp_vqe_bits.ipynb)) parametrized by $a_i$.
      
      We will form a simple line of gates (i.e a rotation acting on qbits (i, i+1)).
    - a layer of jointly parametrized $R_X$ gates (parameter named $b_i$, one gate per qbit)

Hence this family is parametrized by:
- the number of qbits (n)
- the number of alternating layers (depth)

This circuit will not depend on the structure of the Hamiltonian, and will only depends on the number of qbits/spins.

**Question 4**: Write a function `first_family` that generates such a parametrized circuit, given a number of qubit and a circuit depth.


In [ ]:
from qat.lang.AQASM import *

def first_family(nbqbits, depth):
    """TODO"""

In [ ]:
# Quick test
circuit = first_family(3, 1)
assert sum(1 for op in circuit.iterate_simple() if op[0] == "CNOT") == 4\
    or sum(1 for op in circuit.iterate_simple() if op[0] == "RZZ") == 2
circuit = first_family(10, 2)
assert sum(1 for op in circuit.iterate_simple() if op[0] == "CNOT") == 2 * 18\
    or sum(1 for op in circuit.iterate_simple() if op[0] == "RZZ") == 2 * 9
%qatdisplay circuit  --svg
circuit = first_family(5, 1)
%qatdisplay circuit  --svg
circuit = first_family(5, 2)
%qatdisplay circuit  --svg

### Running a VQE

Now, we have all the tools to run our first VQE.

**Question 5:** Use [this section of the documentation](https://myqlm.github.io/running_variational.html#id1), or the helper notebook (last cells), in order to write a function that, given a map representing an Hamiltonian:
- finds the set of parameters that minimize the Hamiltonian's energy using our `first_family` circuits
- and returns the corresponding energy value
We will use the following optimizer parameters:

**method**: COBYLA

**tol**: 1e-4

**options**:{**maxiter**: 300}


In [ ]:

def approximate_ground_state(ham, depth):
    """ TODO """

In [ ]:
# Test
ham = {(0, 1): 1, (1, 2): 1}

print("Hamiltonian:", ham)
ground_energy = exact_ground_energy(ham)
approximate_ground_energy = approximate_ground_state(ham, 2)
print("True ground energy:", ground_energy)
print("Approximate ground energy:", approximate_ground_energy)


**Question 5:** What can we expect from our approximation when we increase the `depth`  parameter of our circuits?
Write a few lines of code to show this behavior on the following Hamiltonian `ham`.

*Note:* You might need to incrase the "maxiter" parameter for larger depth

In [ ]:
ham = {(i, i+1): 1 for i in range(9)}


Now, we only tried to solve instances with a particular shape (where the interactions form a line).

**Question 6:** Propose a simple instance that does not have this symmetry and comment the performances of our first family of circuits on this instance

*hint*: throw in some random interactions, it should break everything

## Matching the Hamiltonian's interactions

We will now move on to the second family.

Circuit in this family will be constructed as follows:
- start with a layer of H gates (one per qubit)
- alternating layers:
    + a layer of RZZ gates (parameter $a_i$), one for each interaction term in the input Hamiltonian
    + a layer of RX gates (parameter $b_i$), one per qubit
    
**Question 7:** Write a function `second_family` that implements this family of circuits, and a function `approximate_ground_state_2` that uses this family.

Then run it on the problematic Hamiltonian from the previous question, and comment on the result.

In [ ]:
def second_family(ham, depth):
    # TODO
    pass



def approximate_ground_state_2(ham, depth):
    # TODO
    pass


Lets break it again!

**Question 8:** Propose a new Hamiltonian that will break this new family.

**hint**: for now, all our Hamiltonians had uniform interaction strengths



Finally, we will design our final family of circuits that should be able to work even for these problematic instances.

Our third and final family of circuits will have exactly the same gate structure as the second family.

**Question 9:** Propose a way to take the interaction strengths into account in the entangling layers of the circuits. Code a function `third_family` that implements this family, and a function `approximate_ground_state_3`.

Comment how it performs on our problematic instance from previous question.



In [ ]:
def third_family(ham, depth):
    # TODO
    pass


def approximate_ground_state_3(ham, depth):
    # TODO
    pass


**Question 10:** Now, lets assume that our Hamiltonian also have local terms:
$$ H = \sum_{i < j} J_{i, j} \sigma_z^i\sigma_z^j + \sum_i h_i \sigma_z^i $$

What modifications would you make to our third family in order to support these new Hamiltonians?


**Answer**: 

# Final comment

This type of parametrized circuits are very standard in the quantum combinatorial optimization literature.

This VQE approach is called Quantum Approximate Optimization Algorithm (**QAOA**). You can find all the details [here](https://arxiv.org/abs/1411.4028).

Roughly, these circuits are built by a parametrized Trotterization of the operator:
$$ exp(-i \alpha H + \beta H_0) $$
where 
$$H_0 = - \sum \sigma_x $$

Hence, we can describe the circuits as:
- preparation of the ground state of $H_0$
- alternatively performing $exp(-i a_i H)$ and $exp(-i b_i H_0)$

The reason why this approach works comes from another field called **Adiabatic Quantum Computing**.